# C: using libraries; stack frames
_COSC 208, Introduction to Computer Systems, 2023-01-30_

## Announcements
* DEI assignment 1 due Thurs, Feb 9 @ 11pm

## Outline
* Warm-up
* Using libraries
* Stack frames

## Warm-up
* In the biological process of cell division, each cell divides into two cells and the process repeats.
* Q1: _Write a function called `cells` that takes the number of rounds of cell division that occur and computes the total number of cells that will exist after the specified number of rounds (assuming you started with a single cell)._

In [7]:
int cells(int rounds) {
    int total = 1;
    for (int i = 0; i < rounds; i++) {
        total = total * 2;
    }
    return total;
}
// Testing code
#include <assert.h>
int main() {
    assert(cells(0) == 1);
    assert(cells(1) == 2);
    assert(cells(8) == 256);
}

<div style="height:20em;"></div>

* Q2: _Write a function called `rounds` that takes the number of cells that should exist and computes the number of rounds of cell division that must occur to have at least that many cells (assuming you started with a single cell)._

In [9]:
int rounds(int cells) {
    int total = 1;
    int rounds = 0;
    while (total < cells) {
        total *= 2;
        rounds++;
    }
    return rounds;
}
// Testing code
#include <assert.h>
int main() {
    assert(rounds(1) == 0);
    assert(rounds(2) == 1);
    assert(rounds(250) == 8);
    assert(rounds(256) == 8);
}

<div style="height:20em;"></div>

🛑 **STOP here** after completing the warm-up; if you have extra time please **skip ahead** to the extra practice.

<div style="page-break-after:always;"></div>

## Using libraries
* Could use exponent to compute number of cells and logarithm to compute number of rounds
* Provided by math library — `#include <math.h>`
    * `double pow(double x, double y);`
    * `double log2(double x);`
* Aside: calculations involving decimal points suffer from small rounding errors, but we are going to using the integer ceiling of the result, so the rounding errors won't be a problem
    * `double round(double x);`
    * `double ceil(double x);`

In [14]:
#include <math.h>
int cells(int rounds) {
    double total = pow(2, rounds);
    total = round(total);
    return (int)total;
} 
int rounds(int cells) {
    double rounds = log2(cells);
    rounds = ceil(rounds);
    return (int)rounds; // Type cast to integer
}
// Testing code
#include <assert.h>
int main() {
    assert(cells(4) == 16);
    assert(rounds(16) == 4);
}

* Compilation error — Why?
```bash
$ gcc -Wall -o cell_division cell_division.c
/usr/bin/ld: /tmp/cell_division-810d67.o: in function `cells':
cell_division.c:(.text+0x1c): undefined reference to `pow'
/usr/bin/ld: /tmp/cell_division-810d67.o: in function `rounds':
cell_division.c:(.text+0x5c): undefined reference to `log2'
clang: error: linker command failed with exit code 1 (use -v to see invocation)
```
* Need to tell compiler to "link" to a library
* Library contains pre-compiled code
* Recall: header file contains function prototypes, which allows compiler to know what functions are available in the library
* At runtime, executable and referenced libraries are loaded into memory

## Program stack

* Stores data for functions that are currently executing
* Stored in random access memory (RAM)
* Composed of stack frames
    * A stack frame contains the values stored in a functions's local variables and formal parameters
    * The size of the stack frame is determined by the number and type of local variables and formal parameters
    * A stack frame is added to the top of the stack when a function is called
    * A stack frame is removed from the top of the stack when a function returns

* Example

In [16]:
#include <stdio.h>
int multiply(int z) {
    return z * 5 / 9;
}
int subtract(int x, int y) {
    return x - y;
}
int convert(int t) {
    int u = subtract(t, 32);
    int v = multiply(u);
    return v;
}
int main() {
    int f = 68;
    int c = convert(68);
    printf("%dF is %dC\n", f, c); 
}

68F is 20C


<div style="height:25em;"></div>

* C is pass-by-value → changes made to a variable inside a function will not be preserved outside of the function

* Q3: _What is the output of this program?_

In [17]:
#include <stdio.h>
int copy(int a, int b) {
    a = b;
    return a;
}
int main() {
    int x = 3;
    int y = 7;
    int z = copy(x, y);
    printf("%d %d %d\n", x, y, z);
}

3 7 7


<div style="page-break-after:always;"></div>

## Extra practice
* Q4: _Write a function called `print_x` that takes an integer n that prints the letter `X` n lines high and n characters wide. You can assume n is an odd number. For example, if n = 9, the program's output would be:_
    ```
    \       /
     \     /
      \   /
       \ /
        X
       / \
      /   \
     /     \
    /       \
    ```

In [23]:
#include <stdio.h>
void print_x_helper(int n, int r) {
    for (int c = 0; c < n; c++) {
        if (c == r) {
            printf("\\");
        }
        else if (c == n-1-r) {
            printf("/");
        }
        else {
            printf(" " );
        }
    }
    printf("\n");
}
void print_x(int n) {
    int r = 0;
    while (r < n/2) {
        print_x_helper(n, r);
        r++;
    }
    for (int c = 0; c < n; c++) {
        if (c == r) {
            printf("X");
        }
        else {
            printf(" ");
        }
    }
    printf("\n");
    r++;
    while (r < n) {
        print_x_helper(n, r);
        r++;
    }
}
// Testing
int main() {
    print_x(3);
    print_x(9);
}

\ /
 X 
/ \
\       /
 \     / 
  \   /  
   \ /   
    X    
   / \   
  /   \  
 /     \ 
/       \


<div style="page-break-after:always;"></div>

* Q5: _Draw the contents of the stack immediately before the program prints "1 x 2"_

In [24]:
#include <stdio.h>
int squared(int base) {
    return base * base;
}
int dbl(int num) {
    printf("%d x 2\n", num);
    return num * 2;   
}
int two(int exponent) {
    int result = 1;
    for (int i = 0; i < exponent; i++) {
        result = dbl(result);
    }
    return result;
}
int main() {
    int n = 3;
    int s = squared(3);
    printf("%d^2 is %d\n", n, s);
    int t = two(3);
    printf("2^%d is %d\n", n, t);
}

3^2 is 9
1 x 2
2 x 2
4 x 2
2^3 is 8


* Use [C tutor](https://pythontutor.com/c.html) to visualize the execution of this program and check your drawing